In [2]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, UpSampling2D, Input
from keras.models import Model, Sequential
#from keras.utils import img_to_array, to_categorical
#from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array


In [3]:

from keras import optimizers
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from os import listdir
from keras.layers import Reshape

In [4]:
root_dir = "C:\\Users\\user\\PycharmProjects\\pythonProject\\datasets\\new"


In [5]:
def convert_img_to_tensor(fpath):
    img = cv2.imread(fpath)
    img = cv2.resize(img, (256, 256))
    return img_to_array(img)

# Load dataset and label data
def get_img_data_and_label(root_dir):
    image_dataset = []
    image_label = []
    classes = listdir(root_dir)
    class_mapping = {c: i for i, c in enumerate(classes)}

    for cls in classes:
        skin_img_list = listdir(f"{root_dir}/{cls}")
        for imgfile in skin_img_list:
            filepath = f"{root_dir}/{cls}/{imgfile}"
            image_dataset.append(convert_img_to_tensor(filepath))
            image_label.append(class_mapping[cls])
    
    return image_dataset, image_label, len(classes), classes

image_dataset, image_labels, no_of_output_layer, classes = get_img_data_and_label(root_dir)

# Normalize and reshape data
image_dataset = np.array(image_dataset, dtype=np.float32) / 255.0
image_dataset = image_dataset.reshape(-1, 256, 256, 3)

In [6]:
input_shape = (256, 256, 3)
autoencoder = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    MaxPooling2D((2, 2), padding='same'),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),

    # Encoder to bottleneck
    Flatten(),
    Dense(64, activation='relu'),  # This will be the bottleneck layer
])

In [7]:
autoencoder.compile(optimizer='adam', loss='relu')


In [8]:
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

# Define input shape
input_img = Input(shape=(256, 256, 3))

# Encoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Load and preprocess images
# resized_images = ...

# Train the autoencoder
# autoencoder.fit(resized_images, resized_images, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


In [9]:
from keras.layers import Input, Conv2D, UpSampling2D, MaxPooling2D
from keras.models import Model

# Encoder
input_img = Input(shape=(256, 256, 3))
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = UpSampling2D((2, 2))(encoded)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Combine encoder and decoder into an autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_7 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                             
                                                                 
 up_sampling2d_1 (UpSampling  (None, 128, 128, 128)    0   

In [10]:
from keras.layers import Input, Conv2D, UpSampling2D, MaxPooling2D
from keras.models import Model

# Encoder
input_img = Input(shape=(256, 256, 3))
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)  # Now shape is (64, 64, 128)

# Decoder
x = UpSampling2D((2, 2))(encoded)  # Now shape is (128, 128, 128)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)  # Now shape is (256, 256, 128)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)  # Now shape is (256, 256, 3)

# Combine encoder and decoder into an autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 256, 256, 64)      1792      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                             
                                                                 
 up_sampling2d_4 (UpSampling  (None, 128, 128, 128)    0   

In [11]:
# Assuming `image_dataset` is your dataset
autoencoder.fit(image_dataset, image_dataset, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
34/34 [==============================] - 577s 17s/step - loss: 0.3634 - val_loss: 0.1386
Epoch 2/10
34/34 [==============================] - 547s 16s/step - loss: 0.2407 - val_loss: 0.1226
Epoch 3/10
26/34 [=====================>........] - ETA: 2:01 - loss: 0.2288

In [ ]:
# Extract encoder model
encoder_model = Model(autoencoder.input, autoencoder.layers[8].output)  # The bottleneck layer is the eighth layer


In [ ]:

# Extract encoded features
encoded_features = encoder_model.predict(image_dataset)


In [ ]:

# Train-test split
xtrain, xtest, ytrain, ytest = train_test_split(encoded_features, image_labels, test_size=0.2, random_state=42)

ytrain = to_categorical(ytrain, no_of_output_layer)
ytest = to_categorical(ytest, no_of_output_layer)


In [ ]:
# Classification model using the autoencoder's encoded features
classification_model = Sequential([
    Dense(64, activation='relu', input_shape=(64,)),
    Dense(no_of_output_layer, activation='softmax')
])

classification_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the classification model
classifier_history = classification_model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest), verbose=1)


In [ ]:
# Evaluate and visualize results
ypred = classification_model.predict(xtest)
ytest_argmax = np.argmax(ytest, axis=1)
ypred_argmax = np.argmax(ypred, axis=1)

print(classification_report(ytest_argmax, ypred_argmax, target_names=classes))


In [ ]:
# Plot training and validation accuracy and loss
acc = classifier_history.history['accuracy']
val_acc = classifier_history.history['val_accuracy']
loss = classifier_history.history['loss']
val_loss = classifier_history.history['val_loss']

epochs = range(len(acc))

In [ ]:

plt.figure()
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


In [ ]:
# Confusion matrix
cm = confusion_matrix(ytest_argmax, ypred_argmax)
sns.heatmap(cm, annot=True, fmt='g', xticklabels=classes, yticklabels=classes)
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.title('Confusion Matrix')
plt.show()